In [3]:
import sys
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'
import ast
import socket
from pymongo import MongoClient
from datetime import datetime
from time import strftime
import json
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils


#Geo hash function  
def getLocation(latitude, longitude):
    location = geo.encode(latitude, longitude, precision=5)
    return location

#Merge two data from different satellites
def merge(AQUA, TERRA):
    
    documents = []
    aquaData = list(AQUA)
    terraData = list(TERRA)
    for aqua in aquaData:
        document = {}
        if document["Location"] != aqua["Location"]:
            document["Location"] = aqua["Location"]
            newAqua = dict(aqua)
            del newAqua["Location"]
            document["AQUA"] = newAqua
        elif document["Location"] == aqua["Location"]:
            newAqua = dict(aqua)
            del newAqua["Location"]
            document["AQUA"] = newAqua
            TERRA = []
            for terra in terraData:
                if terra["Location"] != document["Location"]:
                    document["Location"] = terra["Location"]
                    newTerra = dict(terra)
                    del newTerra["Location"]
                    TERRA.append(newTerra)
                elif terra["Location"] == document["Location"]:
                    #terra["AVEtemperature"] = (aqua["surface_temperature_celcius"]+terra["surface_temperature_celcius"])
                    newTerra = dict(terra)
                    del newTerra["Location"]
                    TERRA.append(newTerra)
                else:
                    print(none)
            document["TERRA"] = TERRA
            documents.append(document)
        return documents

#join two stream
def climate_hotspot_db(climate, hotspot):
    climateData = climate
    hotspotData = hotspot
    noFire = []
    for cli in climateData, hot in hotspotData:
        if cli['Location'] != hot['Location']:
            noFire.append(cli)
            
    return noFire

#send no fire record into MongoDB   
def send(lines):
    client=MongoClient()
    db = client.fit5148_db
    for line in lines:
        db.climate.insert_one(line)
    client.close()

#deal with input data from stream
def transfer(rdd):
    iter = rdd.collect()
    climate = []
    AQUA = []
    TERRA = []
    print(rdd)
    for record in iter:
        data = json.loads(record[1])
        sender_id = data["Senderid"]
        data["Location"] = getLocation(data["latitude"], data["longitude"])
        if sender_id == "climate":
            climate.append(data)
        elif sender_id =="AQUA":# Creat two list (AQUA and TERRA) for merge
            AQUA.append(data)
        elif sender_id == "TERRA":
            TERRA.append(data)
        else:
            raise ValueError("wrong producer")
    
    hotspot = merge(AQUA,TERRA)
    
    records = climate_hotspot_db(climate, hotspot)

    sendToDB(records)



    
if __name__ == '__main__':
    n_secs = 10
    

    conf = SparkConf().setAppName("KafkaStreamProcessor").setMaster("local[2]")
    sc = SparkContext.getOrCreate()
    if sc is None:
        sc = SparkContext(conf=conf)
    sc.setLogLevel("WARN")
    ssc = StreamingContext(sc, n_secs)
    topic = ["Producer1", "Producer2", "Producer3"]
    kafkaStream = KafkaUtils.createDirectStream(ssc, topic, {
                            'bootstrap.servers':'127.0.0.1:9092', 
                            'group.id':'week11-group', 
                            'fetch.message.max.bytes':'15728640',
                            'auto.offset.reset':'largest'})
                        # Group ID is completely arbitrary


ssc.start()

try:
    ssc.awaitTermination(timeout=540)
except KeyboardInterrupt:
    ssc.stop()
    sc.stop()

ssc.stop()
sc.stop()

---------start-------
---------end-------
-------------------------------------------
Time: 2019-05-24 15:46:30
-------------------------------------------
(None, "{'longitude': 142.252, 'latitude': -37.8057, 'Senderid': 'AQUA', 'surface_temperature_celcius': 49, 'confidence': 75, 'Datetime': '2019-05-24T15:46:17'}")
(None, "{'longitude': 145.7491, 'latitude': -36.2825, 'Senderid': 'AQUA', 'surface_temperature_celcius': 47, 'confidence': 73, 'Datetime': '2019-05-24T15:46:17'}")
(None, "{'longitude': 142.8935, 'latitude': -37.3847, 'Senderid': 'AQUA', 'surface_temperature_celcius': 88, 'confidence': 100, 'Datetime': '2019-05-24T15:46:17'}")
(None, "{'surface_temperature_celcius': 121, 'latitude': -37.017, 'Senderid': 'TERRA', 'Datetime': '2019-05-24T15:46:13', 'confidence': 100, 'longitude': 148.1297}")
(None, "{'surface_temperature_celcius': 40, 'latitude': -36.6363, 'Senderid': 'TERRA', 'Datetime': '2019-05-24T15:46:13', 'confidence': 54, 'longitude': 144.3033}")
(None, "{'surface_tem